In [ ]:
%%capture
!pip install datasets
!pip install transformers==4.28.0
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [ ]:
import torchaudio
import librosa
import numpy as np
import random
import pandas as pd
from datasets import load_dataset, load_metric
from datasets import ClassLabel
from IPython.display import display, HTML


In [ ]:
load_csv_dataset = load_dataset('csv', data_files='metadata.csv', sep=',', split='train')
data = load_csv_dataset.train_test_split(test_size=0.2)
train_data = data["train"]
test_data = data["test"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/syakh/.cache/huggingface/datasets/csv/default-fd16d07b4dee002b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


In [ ]:
train_data = data["train"].remove_columns(["file_name"])
test_data = data["test"].remove_columns(["file_name"])

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_data, num_examples=5)

,path,sentence
0,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/083/023/083023_10.wav,عَلَى الْأَرَائِكِ يَنْظُرُونَ
1,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/090/007/090007_23.wav,أَيَحْسَبُ أَنْ لَمْ يَرَهُ أَحَدٌ
2,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/084/003/084003_26.wav,وَإِذَا الْأَرْضُ مُدَّتْ
3,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/085/016/085016_14.wav,فَعَّالٌ لِمَا يُرِيدُ
4,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/092/002/092002_20.wav,وَالنَّهَارِ إِذَا تَجَلَّى


In [ ]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = librosa.resample(np.asarray(speech_array[0]), orig_sr=sampling_rate, target_sr=16000)
    batch["sampling_rate"] = 16000
    batch["target_text"] = batch["sentence"]
    batch["path"] = batch["path"]
    return batch

In [ ]:
train_data = train_data.map(speech_file_to_array_fn)
test_data = test_data.map(speech_file_to_array_fn)

Map:   0%|          | 0/12648 [00:00<?, ? examples/s]

Map:   0%|          | 0/3162 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(train_data, num_examples=10)

,path,sentence,speech,sampling_rate,target_text
0,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/094/004/094004_04.wav,وَرَفَعْنَا لَكَ ذِكْرَكَ,"[0.006124756298959255, 0.020649684593081474, 0.026020336896181107, 0.0306704081594944, 0.03144910931587219, 0.029626389965415, 0.027219133451581, 0.014280910603702068, 0.009788643568754196, 0.014872374013066292, 0.008261139504611492, -0.018612433224916458, -0.05187615379691124, -0.076273113489151, -0.08313991129398346, -0.07787861675024033, -0.08303625881671906, -0.09621582925319672, -0.11162257194519043, -0.11443746089935303, -0.0938028022646904, -0.06834793835878372, -0.0507548488676548, -0.04730191081762314, -0.039584919810295105, -0.021074745804071426, 0.0014719190075993538, 0.022116173058748245, 0.034683503210544586, 0.04313881695270538, 0.03441625460982323, 0.02144608460366726, 0.009491357021033764, 0.009872308000922203, 0.023810116574168205, 0.006812141742557287, -0.022384315729141235, -0.03789007291197777, -0.03188208118081093, -0.021878443658351898, -0.035679832100868225, -0.048183709383010864, -0.05062984302639961, -0.04487084969878197, -0.054612211883068085, -0.08373698592185974, -0.09213441610336304, -0.08009099215269089, -0.0742013156414032, -0.08455415070056915, -0.08681531250476837, -0.07888580858707428, -0.07156801223754883, -0.05700001120567322, -0.040438611060380936, -0.023421525955200195, -0.003965047188103199, 0.014656891115009785, 0.02546708658337593, 0.03438098728656769, 0.04518815129995346, 0.05062631890177727, 0.04979003965854645, 0.03989868611097336, 0.021582387387752533, 0.006362794898450375, 0.007845865562558174, 0.010860075242817402, 0.0004282009322196245, -0.013698097318410873, -0.014193139038980007, -0.003918210975825787, -0.008581678383052349, -0.013651469722390175, -0.009820114821195602, -0.0057492125779390335, -0.0023371607530862093, -0.009646963328123093, -0.010223046876490116, -0.0019000585889443755, -0.0002247431839350611, -0.0006904424517415464, 0.0057642171159386635, 0.021558888256549835, 0.03720400854945183, 0.057523213326931, 0.08400066196918488, 0.10410304367542267, 0.1125655248761177, 0.10829005390405655, 0.09819737076759338, 0.08218103647232056, 0.046362683176994324, 0.0009380141273140907, -0.038911618292331696, -0.07421625405550003, -0.09738828241825104, -0.12267240136861801, -0.15105533599853516, -0.17155638337135315, ...]",16000,وَرَفَعْنَا لَكَ ذِكْرَكَ
1,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/089/015/089015_17.wav,فَأَمَّا الْإِنْسَانُ إِذَا مَا ابْتَلَاهُ رَبُّهُ فَأَكْرَمَهُ وَنَعَّمَهُ فَيَقُولُ رَبِّي أَكْرَمَنِ,"[-5.589216200974079e-09, 7.903089738192648e-09, -8.08840638910624e-09, 9.717640025996843e-09, -1.0223248025909015e-08, 1.1643265018790316e-08, -1.2362812995547756e-08, 1.3401797893664025e-08, -1.3702707413187909e-08, 1.4772854939337776e-08, -1.499454072018125e-08, 1.5591066215847604e-08, -1.600265164825032e-08, 1.552787765035646e-08, -1.5641036910096773e-08, 1.508045777143252e-08, -1.4176197993265305e-08, 1.3572574175668706e-08, -1.0141258499629657e-08, 8.893586311842228e-09, -4.21823997953652e-09, 1.478166922197488e-09, 3.863652509039639e-09, -1.031700413989256e-08, 1.7406495800287303e-08, -2.6374465278422576e-08, 3.589646624391207e-08, -4.653024276990436e-08, 5.91536952754268e-08, -7.075097840925082e-08, 8.378061977509788e-08, -9.453552252125519e-08, 1.0253788218506088e-07, -1.0385922166733508e-07, 9.051809968241287e-08, -4.671434794545348e-08, -6.794583384817088e-08, 3.8230774634939735e-07, -1.5238338164635934e-06, 1.8829770851880312e-05, 3.7061385228298604e-05, 1.3249534276837949e-05, -1.951533477040357e-06, 6.016181828272238e-07, -1.4186046826125676e-07, -7.667050283544086e-08, 1.964311309166078e-07, -2.6553598786449584e-07, 3.0261688266364217e-07, -3.169251669987716e-07, 3.11334872549196e-07, -2.8903130555590906e-07, 2.508216994101531e-07, -1.9695561093158176e-07, 1.2745508115585835e-07, -4.191

In [ ]:
print("Target text:", train_data[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_data[rand_int]["speech"]).shape)
print("Sampling rate:", train_data[rand_int]["sampling_rate"])

Target text: كَلَّا إِنَّهُمْ عَنْ رَبِّهِمْ يَوْمَئِذٍ لَمَحْجُوبُونَ 
Input array shape: (178887,)
Sampling rate: 16000


In [ ]:
from datasets import DatasetDict

data_new = {
    "train": train_data,
    "test": test_data
}

dataset_new = DatasetDict(data_new)
print(dataset_new)

In [ ]:
show_random_elements(train_data, num_examples=10)


,speech,sampling_rate,target_text
0,"[9.169960391063395e-14, -1.127036154289704e-13, -1.188077956328082e-13, -4.0624488907396727e-13, -6.404312301831749e-13, -3.4141596884709746e-13, -6.776429460966793e-13, 4.729633568470448e-13, -6.561914098028587e-13, -2.671883439324396e-13, -9.296722598564969e-14, -5.814633171653816e-13, 3.207824088823086e-13, 1.9088017570636356e-13, -4.86300995132527e-13, 3.5536824134416167e-13, 6.476543476865992e-13, -3.013172935988073e-13, 2.2967492165938685e-13, 8.667296481218445e-13, -5.537103978450753e-13, 1.767501618156475e-13, 3.6212929898672697e-13, -9.5932254653118e-14, 8.57566920036898e-14, 3.8998548856396253e-14, 3.2940878215252656e-13, 1.4519329206812148e-13, 1.3541273137518406e-13, -1.1792951733419557e-14, 6.892708517662605e-13, 4.131132556265543e-13, -2.663486835599582e-13, 3.2228462520239587e-13, 3.233937098147399e-13, 6.008665245048339e-13, 5.675619479603156e-13, 1.2356777927269302e-12, 2.586141478917725e-13, 1.9385613448698324e-13, 9.662427379476118e-14, 1.11072047013442e-13, 1.0799602314856549e-14, 5.398292490105261e-13, 8.260072313637234e-13, 2.442956047957884e-13, -2.6162611573704586e-13, 4.89682079607423e-13, 8.803116655770049e-13, -4.526851712493424e-14, -2.3186270611819104e-13, 4.56231732080814e-13, 5.2400808301864e-13, -3.1480222074942166e-13, -8.728636303328985e-13, 9.874184261036978e-14, -8.006879122397781e-13, -1.0928069430274356e-12, -1.8895094636064286e-13, -4.1276902143679017e-13, -1.1734286502543267e-12, -5.888087868839709e-13, 2.7566162785590265e-13, 7.66453686542462e-15, 8.960998718228474e-14, -7.764093187812016e-13, 2.976389750983244e-13, -1.3375531151410547e-15, -3.5520387629481287e-13, -4.83163923909076e-14, 2.8717401856026906e-14, -1.2929037181141911e-12, 1.1008701545842103e-13, 4.749437605353068e-13, -1.6257260565966797e-13, 3.1372135249861655e-13, 5.664826246976062e-13, -9.425197167872712e-13, -3.5128247966725867e-13, -6.789393266343202e-14, 2.1272168596910002e-13, -8.756075291910248e-13, 1.5091768980346476e-13, 6.460237618292897e-13, 3.3346865850256147e-13, 1.0484058825081799e-13, -1.524104193545428e-13, -1.8073105403741685e-13, 9.765552624364793e-14, -6.491233332100999e-13, 7.689693743958073e-14, -2.5609812122524833e-13, 8.660701821504302e-13, 1.5902304971969483e-13, 2.984741902418986e-13, 5.561754941145469e-13, 6.303862596854226e-13, 1.6332508262495438e-13, 4.044711614248353e-13, 5.788799344388917e-13, ...]",22050,إِنَّهَا عَلَيْهِمْ مُؤْصَدَةٌ
1,"[-3.3422744309064e-05, -0.00014524272410199046, -9.296468488173559e-05, 6.67799804432434e-06, -6.139126344351098e-05, -1.8579203242552467e-06, 4.0579532651463524e-05, -7.9453966463916e-05, -5.998864071443677e-05, 3.3969440664805006e-06, -0.00010627823940012604, -8.160089782904834e-05, -3.240898877265863e-05, -0.00016668756143189967, -0.00017083375132642686, 9.819341357797384e-07, 2.1882853616261855e-05, -5.7047553127631545e-05, 0.00012629630509763956, 0.00010503520024940372, -3.764369830605574e-05, 2.9326829462661408e-05, 1.4343017028295435e-05, 3.6470861232373863e-05, 6.070827294024639e-05, -6.692057650070637e-05, 3.831896901829168e-05, 0.00011023729166481644, -7.911007742222864e-06, 0.00011550759518286213, 0.00016570214938838035, 1.4136006029730197e-05, 8.894166057871189e-06, 7.077768532326445e-05, 9.31892282096669e-06, 5.8864374295808375e-05, 0.0002463435521349311, 0.00016666206647641957, 5.07260556332767e-05, 0.000320852966979146, 0.0003134693833999336, -1.5249870557454415e-06, 1.0003295756177977e-06, 9.48116576182656e-06, -0.00012333269114606082, -0.0001333379914285615, -6.330150063149631e-05, 1.152248660218902e-05, -1.3301088984007947e-05, 6.35956967016682e-05, 6.994455179665238e-05, -6.841499271104112e-05, 0.00011423004616517574, 0.00010740317520685494, -0.00012693159806076437, 7.414536230498925e-05, 0.00014248488878365606, -0.00010117237252416089, -0.00018447219918016344, -0.0002082094142679125, -4.7946436097845435e-05, 3.629342609201558e-05, -1.3803442016069312e-05, 7.33402484911494e-05, 0.00022007411462254822, 8

In [ ]:
dataset_new.save_to_disk("Audio Dataset (Array form)")

Saving the dataset (0/9 shards):   0%|          | 0/12648 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/3162 [00:00<?, ? examples/s]